In [61]:
#ミニマックス法　自分は自分にとって最善手を選び，相手は自分にとって最悪手を選ぶと仮定して最善手を探すアルゴリズム
#三目並べの作成
import random

#ゲームの状態
class State:
  #初期化 自分の石の配置 
  def __init__(self,pieces=None,enemy_pieces=None):
    #石の配置
    self.pieces=pieces if pieces!=None else [0] *9
    self.enemy_pieces=enemy_pieces if enemy_pieces!=None else [0]*9

  #石の数の取得
  def piece_count(self,pieces):
    count=0
    for i in pieces:
      if i==1:
        count+=1
    return count

  #負けかどうか
  def is_lose(self):
    #3並びかどうか
    def is_comp(x,y,dx,dy):
      for k in range(3):
        if y <0 or 2 < y or x < 0 or 2 <x or \
          self.enemy_pieces[x+y*3]==0:
          return False
        x,y=x+dx,y+dy
      return True

    #負けかどうか
    if is_comp(0,0,1,1) or is_comp(0,2,1,-1):
      return True
    for i in range(3):
      if is_comp(0,i,1,0) or is_comp(i,0,0,1):
        return True
    return False

    #引き分けかどうか
  def is_draw(self):
    return self.piece_count(self.pieces)+self.piece_count(self.enemy_pieces)==9

  #ゲーム終了かどうか
  def is_done(self):  
    return self.is_lose() or self.is_draw()

  #次の状態の取得
  def next(self,action):
    pieces=self.pieces.copy()
    pieces[action]=1
    return State(self.enemy_pieces,pieces)

  #合法手のリストの取得
  def legal_actions(self):
    actions=[]
    for i in range(9):
      if self.pieces[i]==0 and self.enemy_pieces[i]==0:
        actions.append(i)
    return actions

  #先手かどうか
  def is_first_player(self):
    return self.piece_count(self.pieces)==self.piece_count(self.enemy_pieces)

  #文字列表示
  def __str__(self):
    ox=('o','x') if self.is_first_player() else ('x','o')
    str=''
    for i in range(9):
      if self.pieces[i]==1:
        str+=ox[0]
      elif self.enemy_pieces[i]==1:
        str+=ox[1]
      else:
        str+='_'
      if i%3==2:
        str+='\n'
    return str


In [62]:
#ランダムで行動選択
def random_action(state):
  legal_actions=state.legal_actions()
  return legal_actions[random.randint(0,len(legal_actions)-1)]

In [63]:
#ランダムとランダムで対戦

#状態の生成
state=State()

#ゲーム終了までのループ
while True:

  #ゲーム終了時
  if state.is_done():
    break;

  #行動の取得
  action=random_action(state)

  #次の状態の取得
  state=state.next(action)

  #文字列表示
  print(state)
  print()

_o_
___
___


_o_
___
__x


oo_
___
__x


oo_
x__
__x


ooo
x__
__x




In [64]:
#stateを渡すと大きいほど勝利確率が高い値を返す
#ゲーム終了ではない時は状態価値を計算し，最大値を返す　合法手ごとの 
#ミニマックス法で状態価値計算
def mini_max(state):
  #負けは状態価値-1
  if state.is_lose():
    return -1

  #引き分けは状態価値0
  if state.is_draw():
    return 0

  #合法手の状態価値の計算
  best_score=-float('inf')
  for action in state.legal_actions():
    score=-mini_max(state.next(action))
    if score>best_score:
      best_score=score

  #合法手の状態価値の最大値を返す
  return best_score


In [65]:
 #ミニマックス法で行動選択
 def mini_max_action(state):
   #合法手の状態価値の計算
   best_action=0
   best_score=-float('inf')
   str=['','']
   for action in state.legal_actions():
     score=-mini_max(state.next(action))
     if score>best_score:
       best_action=action
       best_score=score

     str[0]='{}{:2d},'.format(str[0],action)
     str[1]='{}{:2d},'.format(str[1],score)
   print('action:',str[0], '\nscore:',str[1], '\n')

   #合法手の状態価値の最大値を持つ行動を返す
   return best_action
     

In [66]:
#先手はmini_max_action 後手はrandom_action
#ミニマックス法とランダムで対戦 

#状態の生成
state=State()

#ゲーム終了までのループ
while True:
  #ゲーム終了時
  if state.is_done():
    break

  #行動の取得
  if state.is_first_player():
    action=mini_max_action(state)
  else:
    action=random_action(state)

  #次の状態の取得
  state=state.next(action)

  #文字列表示
  print(state)
  print()


action:  0, 1, 2, 3, 4, 5, 6, 7, 8, 
score:  0, 0, 0, 0, 0, 0, 0, 0, 0, 

o__
___
___


o_x
___
___


action:  1, 3, 4, 5, 6, 7, 8, 
score: -1, 1, 0, 0, 1, 0, 1, 

o_x
o__
___


o_x
ox_
___


action:  1, 5, 6, 7, 8, 
score: -1,-1, 1,-1,-1, 

o_x
ox_
o__


